In [1]:
import xarray as xr
import rydanalysis as ra
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from pathlib import Path
import numpy as np
from tqdm.notebook import tqdm#
from scipy.stats import sem
from scipy.integrate import quad

%matplotlib widget

In [2]:
path = Path.cwd().parent / "raw_data"

live_analysis_path = Path.cwd().parent / "Analysis" 
peaks_file = live_analysis_path / "peaks.h5"

In [3]:
data, peak_df = ra.update_data(path, csv_path="peak_df_2.csv")

Found exisiting peak_df.


2021-04-27 11:39:23.538 INFO    numexpr.utils: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-04-27 11:39:23.538 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


## Analysis

In [4]:
variable = data.ryd_data.variables.columns[0]

summary = ra.summarize_peak_description(peak_df)
counts = summary.counts.to_series()
counts_mean = counts.groupby(variable).mean()
counts_sem = counts.groupby(variable).sem(ddof=1)

In [5]:
fig, ax = plt.subplots()

ax.errorbar(counts_mean.index, counts_mean, counts_sem, linestyle='-', marker='o')
ax.set_xlabel(variable)
ax.set_ylabel("ion counts")

fig.savefig("raw_data.png")
#ax.set_xscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
ampl=float((counts_mean.mean()-counts_mean[0])*2)
off=float(counts_mean.mean())
#print(counts_sem+0.001)
#counts_sem += +0.0001
cosine_model = ra.DampedCosineModel()
cosine_params = cosine_model.make_params()
cosine_params.add("phase", value=180, vary=0)
cosine_params.add("freq", value=4.1, vary=1, max=50, min=0.)
cosine_params.add("decay", value=0.15, vary=1, max=1, min=0.)
cosine_params.add("amp", value=ampl, vary=0, min=0)
cosine_params.add("offset", value=off, vary=0, min=0)
fitres = cosine_model.fit(
    counts_mean.where(counts_mean.index<5).dropna(),
    params=cosine_params,
    x=counts_mean.index.where(counts_mean.index<5).dropna()
)#, scale_covar=0, weights=1/counts_sem)
fitres.plot_residuals()
#print("Amplitude:", (counts_mean.mean()-counts_mean[0])*2)
#print("Offset:", counts_mean.mean())


fig, ax = plt.subplots()
ax.errorbar(counts_mean.index, counts_mean, counts_sem, linestyle='-', marker='o', label="counts")
x_values = np.linspace(counts_mean.index.min(), counts_mean.index.max(), 201)
ax.plot(x_values, fitres.eval(x=x_values))
times = np.linspace(0, 0.2, 110)
plt.text(0.15, 0.95,'Omega: %.5g  $\pm$ %.5g gamma: %.5g  $\pm$ %.5g'%(fitres.params['freq'].value, fitres.params['freq'].stderr, fitres.params['decay'].value, fitres.params['decay'].stderr), transform=ax.transAxes)
ax.set_xlabel(variable+"[us]")
ax.set_ylabel("ion count")

plt.savefig('48P_%s_Fit_Cosine.png'%variable, dpi=600) # add leading zero to the figure name

print("chi^2 reduced", fitres.redchi)
fitres

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

chi^2 reduced 56.24761835898154


In [7]:
1/(4 * fitres.params["freq"])

0.05909791258860264

In [8]:
1.25 - fitres.params["freq"]/2

-0.8651339281669812

In [9]:
fitres.params["freq"]
fitres.plot_residuals()


<AxesSubplot:title={'center':'Model(damped_cosine)'}, xlabel='tmw[us]', ylabel='residuals'>